# Aula 2 - Reinforcement Learning

## Tutorial: Q Learning no ambiente FrozenLake

### Prof. Dr. Ahirton Lopes

# Q* Learning com FrozenLake 4x4

Neste Notebook, implementaremos um agente <b>que reproduz o desafio FrozenLake.</b>

![texto alternativo](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

O objetivo deste jogo é <b>passar do estado inicial (S) para o estado objetivo (G)</b> andando apenas sobre peças congeladas (F) e evitando buracos (H). No entanto, o gelo é escorregadio, **então você nem sempre se moverá na direção pretendida (ambiente estocástico)**

## Pré-requisitos 🏗️

Antes de mergulhar no notebook **você precisa entender**:
- Os fundamentos da aprendizagem por reforço (MC, TD, hipótese de recompensas...)
- Q-aprendizagem

## Etapa 1: Instalando as dependências no Google Colab

In [1]:
!pip install numpy
!pip install gym

     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
      --------------------------------------- 10.2/721.7 kB ? eta -:--:--
     - ----------------------------------- 20.5/721.7 kB 165.2 kB/s eta 0:00:05
     --- --------------------------------- 71.7/721.7 kB 563.7 kB/s eta 0:00:02
     ------ ----------------------------- 122.9/721.7 kB 722.1 kB/s eta 0:00:01
     --------------- ---------------------- 286.7/721.7 kB 1.3 MB/s eta 0:00:01
     ----------------------- -------------- 450.6/721.7 kB 1.7 MB/s eta 0:00:01
     ------------------------------- ------ 593.9/721.7 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 721.7/721.7 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyprojec

## Etapa 0: Importando as dependências 📚

Usamos 3 bibliotecas:

- `Numpy` para nosso Qtable
- `OpenAI Gym` para nosso ambiente FrozenLake
- `Random` para gerar números aleatórios

In [2]:
import numpy as np
import gym
import random

## Passo 1: Criando o ambiente 🎮

- Aqui criaremos o ambiente FrozenLake 8x8.
- OpenAI Gym é uma biblioteca <b> composta por diversos ambientes que podemos usar para treinar nossos agentes.</b>
- No nosso caso optamos por usar Frozen Lake.

In [11]:
env = gym.make("FrozenLake-v1")

## Etapa 2: Criando a tabela Q e inicializando-a 🗄️

- Agora, vamos criar nossa Q-table, para saber de quantas linhas (estados) e colunas (ações) precisamos, precisamos calcular o action_size e o state_size
- OpenAI Gym nos fornece uma maneira de fazer isso: `env.action_space.n` e `env.observation_space.n`

In [12]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [13]:
# Create our Q table with state_size rows and action_size columns (64x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Etapa 3: Criando os hiperparâmetros ⚙️

- Aqui, especificaremos os hiperparâmetros

In [14]:
total_episodes = 500000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob

## Etapa 4: O algoritmo de aprendizagem Q 🧠

- Agora implementamos o algoritmo de aprendizagem Q:
   ![texto alternativo](http://simoninithomas.com/drlc/Qlearning//qtable_algo.png)


In [15]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)


        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward

        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done == True:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)


print ("Pontuação no tempo: " +  str(sum(rewards)/total_episodes))
print(qtable)

Pontuação no tempo: 0.514278
[[3.22547638e-01 6.24036500e-02 5.25059875e-02 5.98512583e-02]
 [3.36197170e-02 5.35046209e-03 1.23967949e-03 1.76986345e-01]
 [1.70978294e-02 4.34640233e-02 1.74164028e-02 2.40112841e-02]
 [4.13863189e-03 1.88466943e-03 7.61854745e-03 2.38824854e-02]
 [2.90682838e-01 1.11956132e-01 6.23416308e-02 4.99146192e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.06111970e-02 1.59072620e-03 1.41083573e-04 3.25863965e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.52875761e-01 3.26108954e-02 1.69641990e-02 7.08624388e-01]
 [4.56918539e-03 4.26825713e-01 1.89131614e-01 4.93091744e-02]
 [8.70034330e-02 1.50730889e-02 2.95943520e-03 6.66333500e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.62167323e-02 1.33389289e-01 7.10101638e-01 1.18636703e-01]
 [2.93379093e-01 9.85021587e-01 3.29093651e-01 2.62591017e-01]
 [0.00000000e+00 0.0000000

## Etapa 5: Usando nossa tabela Q para jogar FrozenLake! 👾

- Após 10.000 episódios, nossa tabela Q pode ser usada como uma "folha de dicas" para jogar FrozenLake"
- Ao executar este celular você poderá ver nosso agente jogando FrozenLake.

In [16]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODIO ", episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, done, info = env.step(action)

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("Chegamos no Objetivo 🏆")
            else:
                print("Caímos em um Obstáculo ☠️")

            # We print the number of step it took.
            print("Número de passos", step)

            break
        state = new_state
env.close()

****************************************************
EPISODIO  0
Chegamos no Objetivo 🏆
Número de passos 33
****************************************************
EPISODIO  1
Chegamos no Objetivo 🏆
Número de passos 51
****************************************************
EPISODIO  2
Chegamos no Objetivo 🏆
Número de passos 13
****************************************************
EPISODIO  3
Caímos em um Obstáculo ☠️
Número de passos 19
****************************************************
EPISODIO  4
Chegamos no Objetivo 🏆
Número de passos 45
